In [202]:
import pandas as pd
import numpy as np
import random
import os

In [203]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [204]:
train = pd.read_csv("C:/Users/jain5/Desktop/open/train.csv")
test = pd.read_csv("C:/Users/jain5/Desktop/open/test.csv")

In [205]:
train.head()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


In [206]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   871393 non-null  object 
 1   User-ID              871393 non-null  object 
 2   Book-ID              871393 non-null  object 
 3   Book-Rating          871393 non-null  int64  
 4   Age                  871393 non-null  float64
 5   Location             871393 non-null  object 
 6   Book-Title           871393 non-null  object 
 7   Book-Author          871393 non-null  object 
 8   Year-Of-Publication  871393 non-null  float64
 9   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 66.5+ MB


In [207]:
train['Book-Author'] = train['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [208]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   871393 non-null  object 
 1   User-ID              871393 non-null  object 
 2   Book-ID              871393 non-null  object 
 3   Book-Rating          871393 non-null  int64  
 4   Age                  871393 non-null  float64
 5   Location             871393 non-null  object 
 6   Book-Title           871393 non-null  object 
 7   Book-Author          871393 non-null  object 
 8   Year-Of-Publication  871393 non-null  float64
 9   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 66.5+ MB


In [209]:
train.head()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,ronajaffe,2001.0,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),williamshakespeare,1981.0,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),walterscott,1981.0,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,sueharrison,1991.0,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,lindalayshuler,1989.0,Signet Book


In [210]:
# Location 분리
train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
train['city'] = train['Location'].apply(lambda x: x.split(',')[0].strip())
train['state'] = train['Location'].apply(lambda x: x.split(',')[1].strip())
train['country'] = train['Location'].apply(lambda x: x.split(',')[2].strip())

train.drop('Location', axis=1, inplace=True)
train = train.replace('na', np.nan)
train = train.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = train['city'].mode()[0]
mode_state = train['state'].mode()[0]
mode_country = train['country'].mode()[0]

# 결측값 대체
train['city'].fillna(mode_city, inplace=True)
train['state'].fillna(mode_state, inplace=True)
train['country'].fillna(mode_country, inplace=True)

print('각 칼럼별 최빈값')
print('국가:', mode_country, '\n주:', mode_state, '\n도시:', mode_city)
# train.isnull().sum()

각 칼럼별 최빈값
국가: usa 
주: california 
도시: n/a


In [211]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
train.loc[(train['Age'] <= 5)|(train['Age'] > 90), 'Age'] = np.nan
train['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
train['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
train['Age'] = train['Age'].fillna(train['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
train['Age_group'] = pd.cut(train['Age'], bins = list(range(5, 91, 5)))
# 시각화로 분포 살펴보기
#train['Age_group'].value_counts().sort_index().plot.bar()
#plt.title('Age-group Distribution (Train)')
#plt.show()

# Age_group2: 이상치 처리 & 미국 노동통계국 자료를 참고한 범주화
train['Age_group2'] = pd.cut(train['Age'], bins = [5, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 91])
# 시각화로 분포 살펴보기
#train['Age_group2'].value_counts().sort_index().plot.bar()
#plt.title('Age-group2 Distribution (Train)')
#plt.show()

In [212]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [213]:
train['Book-Author'] = train['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [214]:
# Title 데이터 정리
import re # 정규표현 모듈

# 정규표현식으로 대소문자 통일, 특수문자 제거
train['Book-Title'] = [re.sub(r'[^0-9a-zA-Z:,]',  ' ',str(i)) for i in train['Book-Title']]
train['Main_Title'] = [i.split('  ')[0] for i in train['Book-Title']]
train['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in train['Book-Title']]
train['Sub_Title'] = np.where(train['Sub_Title'] == '', 'No_SUB', train['Sub_Title'])

In [215]:
# 출판년도 그룹화
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
# train['Year-Of-Publication'] = train['Year-Of-Publication'].fillna(-1)

labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
train['Pub_gb'] = pd.cut(train['Year-Of-Publication'], bins, labels = labels,include_lowest = True)    
train = train.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)

In [216]:
# dataset 바탕화면에 저장
import pandas as pd
import os

os.chdir("C:/Users/jain5/Desktop") # dataset을 저장할 경로를 작성하기
train.to_csv("train_dataset.csv")

-------test data 추가-------

In [217]:
# Title 데이터 정리
import re # 정규표현 모듈

# 소문자로 통일, 특수문자 제거
test['Book-Title'] = test['Book-Title']\
                        .str.lower().replace('[^0-9a-zA-Z]', '', regex=True)
test['Main_Title'] = [i.split('  ')[0] for i in test['Book-Title']]
# SUB_title로 나누기
test['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in test['Book-Title']]
# SUB_title이 없으면 No_SUB
test['Sub_Title'] = np.where(test['Sub_Title'] == '', 'No_SUB', test['Sub_Title'])

In [218]:
# 출판년도 그룹화
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
test['Year-Of-Publication'] = test['Year-Of-Publication'].fillna(-1)
labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
test['Pub_gb'] = pd.cut(test['Year-Of-Publication'], bins, labels = labels,include_lowest = True)    
test = test.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)


In [219]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
test.loc[(test['Age'] <= 5)|(test['Age'] > 90), 'Age'] = np.nan
test['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
test['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
test['Age'] = test['Age'].fillna(test['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
test['Age_group'] = pd.cut(test['Age'], bins = list(range(5, 91, 5)))
# 시각화로 분포 살펴보기
#train['Age_group'].value_counts().sort_index().plot.bar()
#plt.title('Age-group Distribution (Train)')
#plt.show()

# Age_group2: 이상치 처리 & 미국 노동통계국 자료를 참고한 범주화
test['Age_group2'] = pd.cut(test['Age'], bins = [5, 6, 8, 12, 18, 25, 34, 44, 54, 64, 74, 91])
# 시각화로 분포 살펴보기
#train['Age_group2'].value_counts().sort_index().plot.bar()
#plt.title('Age-group2 Distribution (Train)')
#plt.show()

In [220]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
test['Publisher'] = test['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
test['Publisher'] = test['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

In [221]:
# Location 분리
test['Location'] = test['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
test['city'] = test['Location'].apply(lambda x: x.split(',')[0].strip())
test['state'] = test['Location'].apply(lambda x: x.split(',')[1].strip())
test['country'] = test['Location'].apply(lambda x: x.split(',')[2].strip())

test.drop('Location', axis=1, inplace=True)
test = test.replace('na', np.nan)
test = test.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = test['city'].mode()[0]
mode_state = test['state'].mode()[0]
mode_country = test['country'].mode()[0]

# 결측값 대체
test['city'].fillna(mode_city, inplace=True)
test['state'].fillna(mode_state, inplace=True)
test['country'].fillna(mode_country, inplace=True)

print('각 칼럼별 최빈값')
print('국가:', mode_country, '\n주:', mode_state, '\n도시:', mode_city)
# train.isnull().sum()

각 칼럼별 최빈값
국가: usa 
주: california 
도시: toronto


In [222]:
test['Book-Author'] = test['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [226]:
# dataset 바탕화면에 저장
import pandas as pd
import os

os.chdir("C:/Users/jain5/Desktop") # dataset을 저장할 경로를 작성하기
test.to_csv("test_dataset.csv")